In [ ]:
import cv2
import numpy as np
import pandas as pd
from imutils import paths
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image
import os
import tensorflow as tf
from keras.layers import Input

import os
file_list=[]
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/humanos"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/praia"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/obras"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/onibus"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/dino"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/elefante"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/flores"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/cavalos"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/montanhas"))
file_list.append(os.listdir(r"/content/drive/My Drive/Base1/comida"))

# general path
path='/content/drive/My Drive/Base1/'

W=224
H=224

# list of classes
class_names=['humanos', 'praia', 'obras', 'onibus', 'dino', 'elefante', 'flores', 'cavalos', 'montanhas', 'comida'] 

X = []
y = []

# Feature extraction
for classes_files, classe in zip (file_list, range(10)):
    for i in range(100):
      name= str(path) + str(class_names[classe]) + str('/') + str(classes_files[i]) 
      print(name)
      imagem = cv2.imread(name)
      imagem = cv2.resize(imagem,(W,H))  #### <=====
      xd = image.img_to_array(imagem)
      X.append(xd)
      y.append(classe)

# salvando a imagens lidas           
np.save('X.npy', X)

# Saving the classes in a csv file
df_class = pd.DataFrame(y)
df_class.to_csv('y.csv', header=False, index=False)

/content/drive/My Drive/Base1/humanos/0.jpg
/content/drive/My Drive/Base1/humanos/1.jpg
/content/drive/My Drive/Base1/humanos/10.jpg
/content/drive/My Drive/Base1/humanos/12.jpg
/content/drive/My Drive/Base1/humanos/2.jpg
/content/drive/My Drive/Base1/humanos/19.jpg
/content/drive/My Drive/Base1/humanos/13.jpg
/content/drive/My Drive/Base1/humanos/15.jpg
/content/drive/My Drive/Base1/humanos/14.jpg
/content/drive/My Drive/Base1/humanos/20.jpg
/content/drive/My Drive/Base1/humanos/17.jpg
/content/drive/My Drive/Base1/humanos/16.jpg
/content/drive/My Drive/Base1/humanos/11.jpg
/content/drive/My Drive/Base1/humanos/18.jpg
/content/drive/My Drive/Base1/humanos/21.jpg
/content/drive/My Drive/Base1/humanos/22.jpg
/content/drive/My Drive/Base1/humanos/30.jpg
/content/drive/My Drive/Base1/humanos/27.jpg
/content/drive/My Drive/Base1/humanos/28.jpg
/content/drive/My Drive/Base1/humanos/23.jpg
/content/drive/My Drive/Base1/humanos/25.jpg
/content/drive/My Drive/Base1/humanos/26.jpg
/content/driv

In [ ]:
# Rótulos
from tensorflow.keras.utils import to_categorical
y = pd.read_csv('y.csv', header=None)
y=y.to_numpy()
y=np.ravel(y)
y=to_categorical(y)   # transforma cada entrada de y em um vetor  
print(y.shape)

# Imagens
X = np.load('X.npy')
print(X.shape)

(1000, 10)
(1000, 224, 224, 3)


In [ ]:
# Exemplo de Fine-tuning de uma VGG16
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalMaxPooling2D
from keras.applications.vgg16 import VGG16

# Carrega a rede pré-treinada
vgg_conv=VGG16(weights='imagenet', include_top=False, input_shape=(W, H, 3))
# Congelando as camadas carregadas
vgg_conv.trainable=False
# Mostrando a topologia da rede carregada
vgg_conv.summary()
# Adaptando a rede para o problema alvo
model=Sequential()
model.add(vgg_conv)
model.add(Flatten()) #model.add(GlobalMaxPooling2D())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compilando o modelo
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=['accuracy'])
# Plotando o modelo
model.summary()

58889256/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
# Treinamento da rede
from sklearn.model_selection import train_test_split

fp='./bestmodel.hdf5'
mc=tf.keras.callbacks.ModelCheckpoint(
    filepath=fp,
    save_weights_only=True,
    monitor='val_accuracy',
    model='max',
    save_best_only=True,
    verbose=1)

# Separando a base de teste
trainX, testX, trainY, testY=train_test_split(X,y,test_size=.3,
                                              random_state=42,
                                              stratify=y)
# Treinamento do modelo
results=model.fit(trainX, trainY,
                  batch_size=32,
                  epochs=50,
                  verbose=1,
                  validation_split=0.2,
                  callbacks=[mc])

# carregando os pesos do melhor modelo
model.load_weights(fp)




Epoch 1/50
18/18 [==============================] - ETA: 0s - loss: 36.1269 - accuracy: 0.7143
Epoch 1: val_accuracy improved from -inf to 0.92857, saving model to ./bestmodel.hdf5
18/18 [==============================] - 21s 466ms/step - loss: 36.1269 - accuracy: 0.7143 - val_loss: 1.6262 - val_accuracy: 0.9286
Epoch 2/50
18/18 [==============================] - ETA: 0s - loss: 1.5264 - accuracy: 0.9625
Epoch 2: val_accuracy improved from 0.92857 to 0.93571, saving model to ./bestmodel.hdf5
18/18 [==============================] - 5s 292ms/step - loss: 1.5264 - accuracy: 0.9625 - val_loss: 4.0623 - val_accuracy: 0.9357
Epoch 3/50
18/18 [==============================] - ETA: 0s - loss: 1.0585 - accuracy: 0.9821
Epoch 3: val_accuracy did not improve from 0.93571
18/18 [==============================] - 3s 161ms/step - loss: 1.0585 - accuracy: 0.9821 - val_loss: 4.7261 - val_accuracy: 0.9143
Epoch 4/50
18/18 [==============================] - ETA: 0s - loss: 2.8410 - accuracy: 0.9714
Ep

In [ ]:
# Testando a rede treinada
_, acc=model.evaluate(testX, testY, verbose=0)
print('Final Accuracy: %.3f' % (acc * 100.0))


Final Accuracy: 93.667
